# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [143]:
# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [144]:
df = pd.read_csv('previsao_de_renda.csv')
df = df.drop('Unnamed: 0', axis=1)
df.dropna(inplace=True)
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df = pd.get_dummies(df, columns=['tipo_residencia', 'estado_civil', 'educacao', 'tipo_renda', 'sexo','posse_de_veiculo','posse_de_imovel'], drop_first=True)

df.drop('data_ref', axis=1, inplace=True)



In [145]:
df

,id_cliente,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda,tipo_residencia_Casa,tipo_residencia_Com os pais,tipo_residencia_Comunitário,tipo_residencia_Estúdio,...,educacao_Secundário,educacao_Superior completo,educacao_Superior incompleto,tipo_renda_Bolsista,tipo_renda_Empresário,tipo_renda_Pensionista,tipo_renda_Servidor público,sexo_M,posse_de_veiculo_True,posse_de_imovel_True
0,15056,0,26,6.602740,1.0,8060.34,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
1,9968,0,28,7.183562,2.0,1852.15,1,0,0,0,...,0,1,0,0,0,0,0,1,1,1
2,4312,0,35,0.838356,2.0,2253.89,1,0,0,0,...,0,1,0,0,1,0,0,0,1,1
3,10639,1,30,4.846575,3.0,6600.77,1,0,0,0,...,0,1,0,0,0,0,1,0,0,1
4,7064,0,33,4.293151,1.0,6475.97,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12422,11477,0,32,9.849315,2.0,1592.57,1,0,0,0,...,0,0,1,0,1,0,0,1,1,0
12423,16006,0,48,13.887671,1.0,7990.58,1,0,0,0,...,1,0,0,0,1,0,0,0,0,1
12424,6194,0,45,7.832877,2.0,604.82,1,0,0,0,...,0,1,0,0,0,0,0,0,1,1
12425,4922,0,36,4.298630,2.0,3352.27,1,0,0,0,...,0,1,0,0,1,0,0,1,1,0


In [146]:

df.rename(columns={'educacao_Superior completo': 'educacao_Superior_completo'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12427 entries, 0 to 12426
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id_cliente                     12427 non-null  int64  
 1   qtd_filhos                     12427 non-null  int64  
 2   idade                          12427 non-null  int64  
 3   tempo_emprego                  12427 non-null  float64
 4   qt_pessoas_residencia          12427 non-null  float64
 5   renda                          12427 non-null  float64
 6   tipo_residencia_Casa           12427 non-null  uint8  
 7   tipo_residencia_Com os pais    12427 non-null  uint8  
 8   tipo_residencia_Comunitário    12427 non-null  uint8  
 9   tipo_residencia_Estúdio        12427 non-null  uint8  
 10  tipo_residencia_Governamental  12427 non-null  uint8  
 11  estado_civil_Separado          12427 non-null  uint8  
 12  estado_civil_Solteiro          12427 non-null 

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [147]:
from sklearn.metrics import mean_squared_error
train, test = train_test_split(df, test_size=0.25, random_state=0)
test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3107 entries, 5018 to 2431
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id_cliente                     3107 non-null   int64  
 1   qtd_filhos                     3107 non-null   int64  
 2   idade                          3107 non-null   int64  
 3   tempo_emprego                  3107 non-null   float64
 4   qt_pessoas_residencia          3107 non-null   float64
 5   renda                          3107 non-null   float64
 6   tipo_residencia_Casa           3107 non-null   uint8  
 7   tipo_residencia_Com os pais    3107 non-null   uint8  
 8   tipo_residencia_Comunitário    3107 non-null   uint8  
 9   tipo_residencia_Estúdio        3107 non-null   uint8  
 10  tipo_residencia_Governamental  3107 non-null   uint8  
 11  estado_civil_Separado          3107 non-null   uint8  
 12  estado_civil_Solteiro          3107 non-null 

In [161]:
X = train.drop('renda', axis=1)
y = train['renda']



In [162]:
# Regularization
coefs = []
alphas = [0,0.001,0.005,0.01,0.05,0.1]
md = sm.OLS(y,X, hasconst=True)
for i in alphas:
    reg1 = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0.01
                         , alpha = i)
    coefs.append(reg.rsquared)
    print(coefs)
    print(i)
                         

coefs

[0.25171794350701693]
0
[0.25171794350701693, 0.25171794350701693]
0.001
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.005
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.01
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.05
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.1


[0.25171794350701693,
 0.25171794350701693,
 0.25171794350701693,
 0.25171794350701693,
 0.25171794350701693,
 0.25171794350701693]

In [163]:
coefs = []
alphas = [0,0.001,0.005,0.01,0.05,0.1]
md = sm.OLS(y,X, hasconst=True)
for i in alphas:
    reg2 = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = i)
    coefs.append(reg.rsquared)
    print(coefs)
    print(i)

[0.25171794350701693]
0
[0.25171794350701693, 0.25171794350701693]
0.001
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.005
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.01
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.05
[0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693, 0.25171794350701693]
0.1


In [155]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo_M                         with p-value 8.31818e-250
#############
['tempo_emprego', 'sexo_M']
Add  idade                          with p-value 2.84326e-06
#############
['tempo_emprego', 'sexo_M', 'idade']
Add  tipo_renda_Empresário          with p-value 1.71608e-05
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário']
Add  educacao_Superior_completo     with p-value 0.00047602
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário', 'educacao_Superior_completo']
Add  qt_pessoas_residencia          with p-value 0.00651349
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário', 'educacao_Superior_completo', 'qt_pessoas_residencia']
#############
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empresário', 'educacao_Superior_completo', 'qt_pessoas_residencia']
resulting features:
['tempo_emprego', 'sexo_M', 'idade', 'tipo_renda_Empre

In [164]:
coefs = []
alphas = [0,0.001,0.005,0.01,0.05,0.1]
modelo = 'renda ~ tempo_emprego + sexo_M + idade + tipo_renda_Empresário + educacao_Superior_completo + qt_pessoas_residencia'      
md = smf.ols(modelo, data = train)
for i in alphas:
    reg3 = md.fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = i)
    coefs.append(reg.rsquared_adj)
    print(coefs)
    print(i)

[0.2499472427172088]
0
[0.2499472427172088, 0.2499472427172088]
0.001
[0.2499472427172088, 0.2499472427172088, 0.2499472427172088]
0.005
[0.2499472427172088, 0.2499472427172088, 0.2499472427172088, 0.2499472427172088]
0.01
[0.2499472427172088, 0.2499472427172088, 0.2499472427172088, 0.2499472427172088, 0.2499472427172088]
0.05
[0.2499472427172088, 0.2499472427172088, 0.2499472427172088, 0.2499472427172088, 0.2499472427172088, 0.2499472427172088]
0.1


In [165]:
X_test = test.drop('renda', axis=1)
y_test = test['renda'] 

In [172]:
from sklearn.metrics import r2_score
y_pred = reg1.predict(X_test)

n = len(X_test)
p = len(X.columns)

R = r2_score(y_test, y_pred)
print(R)
Adj_r2 = 1 - ((1-R) * (n-1)/(n-p-1))
print(Adj_r2)


0.2698447726576798
0.2639201116114097


In [173]:
from sklearn.metrics import r2_score
y_pred = reg2.predict(X_test)

n = len(X_test)
p = len(X.columns)

R = r2_score(y_test, y_pred)
print(R)
Adj_r2 = 1 - ((1-R) * (n-1)/(n-p-1))
print(Adj_r2)


0.26991041466653976
0.26398628625584963


In [174]:
from sklearn.metrics import r2_score
y_pred = reg3.predict(X_test)

n = len(X_test)
p = len(X.columns)

R = r2_score(y_test, y_pred)
print(R)
Adj_r2 = 1 - ((1-R) * (n-1)/(n-p-1))
print(Adj_r2)


0.26954359475315404
0.2636164898744876


O modelo apos o stepwise obteve o melhor R quadradod.

O Modelo 2 teve o melhor R quadrado.